In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from IPython.display import clear_output

In [2]:
# Useful functions for formatting data sets

def lon_to_longitude(df):
    '''
    Simple function to add latitude and longitude columns if dataframe only contains columns labeled lon and lat
    '''
    if 'lon' in df.columns:
        df['longitude'] = df['lon']
        df.drop('lon', axis = 1, inplace = True)
    
    if 'lat' in df.columns:
        df['latitude'] = df['lat']
        df.drop('lat', axis = 1, inplace = True)
    

def mv_rounder(df, feature = 'none', date = 'none'):
    '''
    Simple function to round latitude and longitude to nearest 0.5, and drop duplicates (averaging value in feature between duplicates).
    Note that latitude and longitude columns must be named 'latitude' and 'longitude' not 'lat' and 'lon'.

    inputs:

    df (Pandas DataFrame) - dataframe to be used
    feature (column name) - if set, feature to average when duplicates dropped
    data (column name) - if set, date column to be used for dropping duplicates
    '''

    if ('latitude' and 'longitude') in df.columns:
        df['longitude'] = df['longitude'].apply(lambda x: round(x/0.5) * 0.5).apply(lambda x: round(x,1))
        df['latitude'] = df['latitude'].apply(lambda x: round(x/0.5) * 0.5).apply(lambda x: round(x,1))

        if date == 'none':
            if feature != 'none':
                df = df.groupby(['latitude', 'longitude']).agg({feature: 'mean'}).reset_index()
            else:
                df.drop_duplicates(subset = ['latitude','longitude'], inplace = True)

        else:
            if feature != 'none':
                df = df.groupby(['latitude', 'longitude', date]).agg({feature: 'mean'}).reset_index()
            else:
                df.drop_duplicates(subset = ['latitude','longitude', date], inplace = True)           
        
        return df

In [3]:
# Import fire data

# CSV file names
fire_folder_path = './USA_fire_date_2010_2023'
csv_fire_files = [file for file in os.listdir(fire_folder_path) if file.endswith('.csv')]

# Loop through each CSV file and create a dataframe for said file, restricting to rough california coordinates
fire_dataframes = {}
for csv_file in csv_fire_files:

    year = int(csv_file.split('_')[2].split('.')[0])
    
    fire_df = pd.read_csv(os.path.join(fire_folder_path, csv_file), usecols = ['latitude','longitude','acq_date'])
    mv_rounder(fire_df, date = 'acq_date')
    fire_df.rename(columns = {'acq_date':'date'}, inplace = True)
    fire_df['fire'] = [1]*len(fire_df)
    
    
    fire_dataframes[f'{year}'] = fire_df

fire_2015 = fire_dataframes['2015']
fire_2015['month'] = fire_2015['date'].apply(lambda x: x[5:7])
len(fire_2015[fire_2015['month'] == '08'])

275

In [4]:
# Import humidity data

# CSV file names
humidity_folder_path = './humidity_data/processed_data'
csv_humidity_files = [file for file in os.listdir(humidity_folder_path) if file.endswith('.csv')]

# Loop through each CSV file and create a dataframe for said file, restricting to rough california coordinates
humidity_dataframes = {}
for csv_file in csv_humidity_files:

    day = csv_file.split('.')[0]
    
    humidity_df = pd.read_csv(os.path.join(humidity_folder_path, csv_file))
    
    lon_to_longitude(humidity_df)
    mv_rounder(humidity_df)
    humidity_df = humidity_df[humidity_df['Qair_f_inst'] != 0.0]
    humidity_df['date'] = [day]*len(humidity_df)
    humidity_df.drop('spatial_ref', axis = 1, inplace = True)
    
    humidity_dataframes[f'{day}'] = humidity_df

humidity_dataframes['2015-08-11']
humidity_all_data = pd.concat(humidity_dataframes, ignore_index=True)
humidity_all_data

,Qair_f_inst,longitude,latitude,date
0,0.011474,-117.0,32.5,2015-08-11
1,0.009796,-116.5,32.5,2015-08-11
2,0.008041,-116.0,32.5,2015-08-11
3,0.011642,-117.0,33.0,2015-08-11
4,0.009769,-116.5,33.0,2015-08-11
...,...,...,...,...
5234,0.006133,-122.0,42.0,2015-08-20
5235,0.005810,-121.5,42.0,2015-08-20
5236,0.005783,-121.0,42.0,2015-08-20
5237,0.005644,-120.5,42.0,2015-08-20


In [5]:
# Import temperature data

# CSV file names
temperature_folder_path = './temperature_data/processed'
csv_temperature_files = [file for file in os.listdir(temperature_folder_path) if file.endswith('.csv')]

# Loop through each CSV file and create a dataframe for said file, restricting to rough california coordinates
temperature_dataframes = {}
for csv_file in csv_temperature_files:

    day = csv_file.split('.')[0]
    
    temperature_df = pd.read_csv(os.path.join(temperature_folder_path, csv_file), index_col = 'index')
    
    lon_to_longitude(temperature_df)
    mv_rounder(temperature_df)
    temperature_df = temperature_df.dropna()
    temperature_df.drop('spatial_ref', axis = 1, inplace = True)
    temperature_df.rename(columns = {'time':'date'}, inplace = True)
    
    temperature_dataframes[f'{day}'] = temperature_df

temperature_dataframes['2015-08-11']
temperature_all_data = pd.concat(temperature_dataframes, ignore_index=True)

In [6]:
# Import wind data

# CSV file names
wind_folder_path = './wind_data/wind_data/csv/monthly'
csv_wind_files = [file for file in os.listdir(wind_folder_path) if file.endswith('.csv')]

# Loop through each CSV file and create a dataframe for said file, restricting to rough california coordinates
wind_dataframes = {}
for csv_file in csv_wind_files:

    day = csv_file.split('.')[0]
    
    wind_df = pd.read_csv(os.path.join(wind_folder_path, csv_file), usecols = ['SPEEDLML','lon','lat'])
    
    lon_to_longitude(wind_df)
    mv_rounder(wind_df)
    wind_df = wind_df[wind_df['SPEEDLML'] != 0]
    wind_df['date'] = [day]*len(wind_df)
    
    wind_dataframes[f'{day}'] = wind_df

wind_dataframes['2015-08-11']
wind_all_data = pd.concat(wind_dataframes, ignore_index=True)
wind_all_data

,SPEEDLML,longitude,latitude,date
0,4.758888,-117.0,33.0,2015-08-11
1,7.541190,-116.0,33.0,2015-08-11
2,5.481746,-115.5,33.0,2015-08-11
3,5.193692,-115.0,33.0,2015-08-11
4,3.147652,-117.5,33.5,2015-08-11
...,...,...,...,...
4118,4.231195,-120.5,41.5,2015-08-20
4119,4.969471,-120.0,41.5,2015-08-20
4120,3.717394,-123.0,42.0,2015-08-20
4121,3.906278,-122.5,42.0,2015-08-20


In [7]:
# Import precipitation data

# CSV file names
precipitation_folder_path = './precipitation_data/.csv/daily'
csv_precipitation_files = [file for file in os.listdir(precipitation_folder_path) if (file.endswith('.csv') and file.startswith('2015'))]

# Loop through each CSV file and create a dataframe for said file, restricting to rough california coordinates
precipitation_dataframes = {}
for csv_file in csv_precipitation_files:

    day = csv_file.split('.')[0]
    
    precipitation_df = pd.read_csv(os.path.join(precipitation_folder_path, csv_file), usecols = ['precipitationCal','lon','lat'])
    
    lon_to_longitude(precipitation_df)
    mv_rounder(precipitation_df)
    precipitation_df = precipitation_df[precipitation_df['precipitationCal'] != 0]
    precipitation_df['date'] = [day]*len(precipitation_df)
    
    precipitation_dataframes[f'{day}'] = precipitation_df

precipitation_dataframes['2015-08-11']
precipitation_all_data = pd.concat(precipitation_dataframes, ignore_index=True)

In [26]:
# Merge dataframes into one

# Precipitaion merge on left as otherwise it creates extra fire events, fire merge on left to preserve points of no fire
df_total = pd.merge(humidity_all_data, temperature_all_data, on=['latitude', 'longitude','date'], how = 'outer')
df_total = pd.merge(df_total,precipitation_all_data, on=['latitude', 'longitude','date'], how = 'left')
df_total = pd.merge(df_total,wind_all_data, on=['latitude', 'longitude','date'], how = 'outer')
df_total = pd.merge(df_total, fire_dataframes['2015'], on=['latitude', 'longitude','date'], how = 'left')

df_total.rename(columns = {'Qair_f_inst':'humidity', 'AvgSurfT_tavg':'temperature', 'precipitationCal':'precipitation', 'SPEEDLML':'wind_speed'}, inplace = True)

df_total.replace(np.nan,0, inplace = True)
df_total.drop_duplicates(subset = ['latitude','longitude', 'date'], inplace = True)
df_total.drop('month', axis = 1, inplace = True)

print(sum(df_total['fire']))
print(len(fire_2015[fire_2015['month'] == '08']))
df_total

272.0
275


,humidity,longitude,latitude,date,temperature,precipitation,wind_speed,fire
0,0.011474,-117.0,32.5,2015-08-11,296.54483,0.0,0.000000,0.0
1,0.009796,-116.5,32.5,2015-08-11,296.70218,0.0,0.000000,0.0
2,0.008041,-116.0,32.5,2015-08-11,302.85245,0.0,0.000000,0.0
3,0.011642,-117.0,33.0,2015-08-11,296.78555,0.0,4.758888,0.0
4,0.009769,-116.5,33.0,2015-08-11,296.72223,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...
5668,0.000000,-123.0,38.5,2015-08-20,0.00000,0.0,4.923879,0.0
5669,0.000000,-124.0,39.5,2015-08-20,0.00000,0.0,3.997697,0.0
5670,0.000000,-124.0,40.0,2015-08-20,0.00000,0.0,3.725361,0.0
5671,0.000000,-124.5,40.5,2015-08-20,0.00000,0.0,6.733368,0.0


In [27]:
df_total.to_csv('./ML_dataframe.csv', index = False)

In [9]:
# ----------------------------------------------------
#-----------------------------------------------------
#-----------------------------------------------------